In [1]:
import torch
from torch import Tensor
import csv
from operator import itemgetter

torch.set_default_dtype(torch.float64)

data_dir = "../Data/"

### **Read in the data.**

In [2]:
with open(data_dir+"raman_data_index_july29.csv", newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    idx_content = []
    for row in reader:
        idx_content.append(row)

# Get rid of bad runs
idx_content = [d for d in idx_content if d["filename"].find("bad_run") == -1]

h20_only = [d for d in idx_content if d["molefraction_water"] == "1"]
h20_tmma = [d for d in idx_content if d["molefraction_hno3"] == "0" and 
                                      d["molefraction_tmma"] != "0"]
h20_hno3 = [d for d in idx_content if d["molefraction_hno3"] != "0" and 
                                      d["molefraction_tmma"] == "0"]
all_comps = [d for d in idx_content if d["molefraction_hno3"] != "0" and 
                                      d["molefraction_tmma"] != "0"]

print(len(h20_only), len(h20_tmma), len(h20_hno3), len(all_comps))

9 18 17 72


### **Make a basic tensor array holding the 'all_comps' data.**

In [8]:
# Sort compositions lexicographically
csrt = [(float(d["molefraction_tmma"]), 
         float(d["molefraction_hno3"]), 
         float(d["molefraction_water"]), d) for d in all_comps ]
csrt = sorted(csrt, key=itemgetter(0,1,2))
all_comps = [d for _, _, _, d in csrt]

# Compositions to centered logs
def c2cl(vec):
    res = torch.log(vec)
    res = res - res.mean()
    return res[:-1]

# Centered logs to compositions
def cl2c(vec):
    s = -torch.sum(vec, (0,), keepdim=True)
    res = torch.cat((vec, s))
    res = torch.exp(res)
    res = res / res.sum()
    return res

# Formatted id string
def mkidstr(idlist):
    idstr = "["
    for id in idlist:
        idstr += f"{id:2d},"
    lid = len(idstr)
    idstr = idstr[:lid-1] + "]"
    return idstr

# Make the array
spec = []
expts = []
for d in all_comps:

    c = torch.zeros(3, dtype=torch.float64)
    c[0] = float(d["molefraction_tmma"])
    c[1] = float(d["molefraction_hno3"])
    c[2] = float(d["molefraction_water"])
    # Move to centered logs, drop redundant water concentration
    c = c2cl(c)

    wavenos = []
    with open("../Data/raw_data/"+d["filename"]+".txt", 'r') as datafile:
        for line in datafile.readlines():
            words = line.split()
            wavenos.append(float(words[0]))
            spec.append(float(words[1]))
    lw = len(wavenos)
    wavenos = Tensor(wavenos)
    wavenos = wavenos.unsqueeze(-1)
    c = c.tile((lw,1))
    pts = torch.hstack((wavenos, c))
    expts.append(pts)

spec = Tensor(spec)
specsize = pts.shape # I checked, they are all the same
#print(specsize)
expts = torch.stack(expts)
#print(expts.shape)
n_expts = expts.shape[0]
spec = spec.reshape(n_expts, -1)
print(expts.shape)
print(spec.shape)

mf = [(float(d["molefraction_tmma"]), 
       float(d["molefraction_hno3"]), 
       float(d["molefraction_water"]) ) for d in all_comps]
lf = [tuple(e[0,1:].tolist()) for e in expts]
efile = data_dir+"all_comps_order.txt"
cl_thresh = 0.025
clusters = []
with open(efile, "w") as efd:
    for en in enumerate(zip(mf, lf)):
        efd.write(repr(en)+"\n")

        vec = Tensor(en[1][1]).unsqueeze(-1)
        id = en[0]
        found = False
        for c in clusters:
            cv = c[1].mean(dim=-1, keepdims=True)
            disc = torch.linalg.vector_norm(cv-vec)/torch.linalg.vector_norm(cv)
            if disc < cl_thresh:
                found = True
                c[1] = torch.hstack((c[1], vec))
                c[0].append(id)
                break
        if not found:
            c = [[id], vec]
            clusters.append(c)

    clens = Tensor([len(c[0]) for c in clusters]).to(int)

    line = f"Found {len(clusters)} clusters"
    print(line)
    efd.write(line + "\n")
    for l in range(clens.min(), clens.max()+1):
        num =len(clens[clens==l])
        line = f"Found {num} clusters of length {l}"
        print(line)
        efd.write(line + "\n")

    efd.write("Clusters:\n")
    clid = 0
    for c in clusters:
        idstr = mkidstr(c[0])
        lid = len(idstr)
        lspc = 15 - lid
        spc = ""
        for _ in range(lspc): spc += " " 
        clm = c[1].mean(dim=-1)
        cm = tuple(cl2c(clm).tolist())
        c.append(tuple(clm.tolist())) ; c.append(cm)
        cstr = "[{0:7.5f}, {1:7.5f}, {2:7.5f}]".format(*cm)
        line = f"{clid:2d} " +idstr + spc + cstr
        print(line)
        efd.write(line + "\n")
        clid += 1

dt = {"expts": expts, "spec": spec}
torch.save(dt, data_dir+"full_spec_all_comps.pt")

torch.Size([72, 3888, 3])
torch.Size([72, 3888])
Found 29 clusters
Found 5 clusters of length 1
Found 5 clusters of length 2
Found 19 clusters of length 3
 0 [ 0, 1, 2]     [0.01000, 0.03000, 0.96000]
 1 [ 3, 4, 5]     [0.01100, 0.08296, 0.90604]
 2 [ 6, 7]        [0.01101, 0.13907, 0.84992]
 3 [ 8, 9,10]     [0.01266, 0.21226, 0.77508]
 4 [11,12,13]     [0.01300, 0.27491, 0.71210]
 5 [14,15,16]     [0.01366, 0.36212, 0.62421]
 6 [17]           [0.01700, 0.03200, 0.95100]
 7 [18,19,20]     [0.02201, 0.03168, 0.94630]
 8 [21,22,23]     [0.02300, 0.06600, 0.91100]
 9 [24,25,32]     [0.02432, 0.10491, 0.87076]
10 [26,27,28]     [0.02400, 0.12133, 0.85467]
11 [29,30,31]     [0.02401, 0.14138, 0.83461]
12 [33,34]        [0.02601, 0.21661, 0.75738]
13 [35,36,37]     [0.02700, 0.26400, 0.70900]
14 [38,39,40]     [0.02900, 0.35600, 0.61500]
15 [41,42,43]     [0.03602, 0.03435, 0.92962]
16 [44,47]        [0.03650, 0.03850, 0.92501]
17 [45]           [0.03600, 0.04400, 0.92000]
18 [46]          

### **ROI Extraction.**

In [4]:
# These bounds produce ROI sample numbers [168, 79, 162, 167] across all spectra 
roi_bds = Tensor([[889.89, 1090.0], [1270.0, 1360.0], [1379.99, 1560.0], [1589.95, 1770.0]])
roi_ids = ["Nitrate", "Nitric Acid", "C-N", "C=O"]

expts_roi = [] ; spec_roi = []
for i in range(n_expts):
    e = expts[i] ; s = spec[i]

    mask = torch.zeros_like(e[:,0])
    for r in roi_bds:
        m = torch.logical_and(e[:,0]>r[0], e[:,0]< r[1])
        mask = torch.logical_or(mask, m)
    expts_roi.append(e[mask]) ; spec_roi.append(s[mask])

expts_roi = torch.stack(expts_roi)
spec_roi = torch.stack(spec_roi)
print(expts_roi.shape, spec_roi.shape)

dt = {"expts_roi": expts_roi, "spec_roi": spec_roi}
torch.save(dt, data_dir+"roi_all_comps.pt")


torch.Size([72, 576, 3]) torch.Size([72, 576])


### **Get some summary statistics useful for initialization**

In [5]:
global_mean = spec_roi.mean()
inspec_vars = spec_roi.var(dim=-1)
inspec_means = spec_roi.mean(dim=-1)
mean_inspec_var = inspec_vars.mean()

mvom = 0.0; nvom = 0
hstr = """
Cluster IDs           Mean Comp.                Mean CL Comp.      Mean I.S. Var Var. of Sp. Means"""
print(hstr)
for c in clusters:
    ids = c[0]
    mvar = inspec_vars[ids].mean()
    c.append(mvar)
    c_varofmeans = 0.0
    if len(ids) != 1:
        c_varofmeans = inspec_means[ids].var()
        mvom += c_varofmeans
        nvom += 1
    c.append(c_varofmeans)
    
    idstr = mkidstr(c[0])
    lid = len(idstr)
    
    lspc = 15 - lid
    spc = ""
    for _ in range(lspc): spc += " " 
    clstr = "[{0:8.5f}, {1:8.5f}]".format(*c[2])
    cstr = "[{0:7.5f}, {1:7.5f}, {2:7.5f}]".format(*c[3])
    mvarstr = f"{mvar:12.5E}"
    vomstr = f"{c_varofmeans:12.5E}"
    line = f"{c[0][0]:2d} " +idstr + spc + cstr + "  " + clstr + mvarstr + "  " + vomstr
    print(line)
mvom /= nvom

print(f"Global mean = {global_mean:12.5E}")
print(f"Mean in-spectrum variance = {mean_inspec_var:12.5E}")
print(f"Mean Variance-of-means = {mvom:12.5E}")



Cluster IDs           Mean Comp.                Mean CL Comp.      Mean I.S. Var Var. of Sp. Means
 0 [ 0, 1, 2]     [0.01000, 0.03000, 0.96000]  [-1.88765, -0.78904] 5.24773E+07   1.00134E+06
 3 [ 3, 4, 5]     [0.01100, 0.08296, 0.90604]  [-2.14410, -0.12330] 1.92367E+08   3.45998E+06
 6 [ 6, 7]        [0.01101, 0.13907, 0.84992]  [-2.29444,  0.24213] 3.45982E+08   1.14950E+06
 8 [ 8, 9,10]     [0.01266, 0.21226, 0.77508]  [-2.31132,  0.50808] 3.07737E+08   3.96833E+05
11 [11,12,13]     [0.01300, 0.27491, 0.71210]  [-2.35181,  0.70002] 2.38635E+08   2.05153E+06
14 [14,15,16]     [0.01366, 0.36212, 0.62421]  [-2.36636,  0.91093] 2.62738E+08   5.29272E+06
17 [17]           [0.01700, 0.03200, 0.95100]  [-1.55227, -0.91975] 2.78509E+08   0.00000E+00
18 [18,19,20]     [0.02201, 0.03168, 0.94630]  [-1.37499, -1.01088] 6.17642E+07   2.39122E+05
21 [21,22,23]     [0.02300, 0.06600, 0.91100]  [-1.57774, -0.52358] 2.13129E+08   2.81168E+06
24 [24,25,32]     [0.02432, 0.10491, 0.87076]  [-1.679

In [6]:
# roi_max = 1770.0 ; roi_min = 889.90
# dk = expts[:,:-1,0] - expts[:,1:,0]
# for e in range(expts.shape[0]):
#     rnge = torch.logical_and(expts[e,:,0] > roi_min, expts[e,:,0] < roi_max)[:-1]
#     ddk = dk[e,rnge]
#     print(e, ddk.max().item(), ddk.min().item(), ddk.mean().item())

# rnge0 = torch.logical_and(expts[0,:,0] > roi_min, expts[0,:,0] < roi_max)[:-1]
# mean_spacing = dk[:,rnge0].mean().item()
# print (mean_spacing, int((roi_max - roi_min)/mean_spacing))

# lf = Tensor(lf)
# min1 = lf[:,0].min().item() ; max1 = lf[:,0].max().item()
# min2 = lf[:,1].min().item() ; max2 = lf[:,1].max().item()
# grid_bounds = ((roi_min, roi_max), (min1, max1), (min2, max2))
# print(grid_bounds)